# Land Use History and Fire

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:


Outputs:




In [2]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

_, lulc = desired_lulc()

# age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

KeyboardInterrupt: 

In [ ]:
aggregate_lu_types = False
history_interval = 15

name = ""
name_lu = f"{name}_{'aggregated' if aggregate_lu_types else 'non_aggregated'}"
name_lu += f"_{history_interval}yr" if history_interval > 0 else "_all"

## Land Use Land Cover

### The MapBiomas Collection 9 land use/land cover data is mapped to the following classes:

- 3 <- Forest Formation
- 6 <- Floodable Forest 
- 15 <- Pasture
- 20 <- Sugar Cane
- 21 <- Mosaic of Uses
- 35 <- Palm Oil
- 39 <- Soybean
- 40 <- Rice
- 41 <- Other temporary crops
- 46 <- Coffee
- 47 <- Citrus
- 48 <- Other perennial crops
- 62 <- Cotton

### Grouping land use types (if aggregation is desired)
- 10 <- pasture (15)
- 20 <- perennial crops (46, 47, 35, 48)
- 30 <- annual crops (39, 20, 40, 62, 41)
- 40 <- mosaic (21)


In [42]:


# Listing land use types that are NOT forest
if (aggregate_lu_types):
    
    lu_index = [10,20,30,40]
    
    # A flat list of pixel values to replace.
    from_list = [15, 46, 35, 46, 47, 48, 39, 20, 40, 41, 62, 21, 3, 6]

    # A corresponding list of new values.
    to_list = [10, 20, 20, 20, 20, 20, 30, 30, 30, 30, 30, 40, 3, 6]

    # for each forest age, mask the lulc of the year immediately preceding abandonment
    lulc_aggregated = ee.Image()
    for band in lulc.bandNames().getInfo():
        lu_yr = lulc.select(band)
        remapped_band = lu_yr.remap(from_list, to_list).rename(band)
        lulc_aggregated = lulc_aggregated.addBands(remapped_band)

    lulc = lulc_aggregated.slice(1).byte()
else:
    lu_index = [15, 46, 35, 46, 47, 48, 39, 20, 40, 41, 62, 21, 3]



### Restricting history to n years before abandonment

Since we only have land use history dating back to 1985, the data is censored. In order to restrict history to exclude pixels with possible long histories of anthropogenic use pre-1985 which could not be included in the analysis:

- Make a mask with regrowing pixels that were forested in 1985 (keep only the patches that were forested in 1985, then subsequently deforested, and regrew after that).

- Making a mask (cutoff year) including the "history_interval" years before detected regrowth

![note](intervals.png)
Illustration of interval considered for analysis with history_interval

In [ ]:
# for each year, make a mask for all anthropogenic pixels, and multiply by the year to have the
# pixel value be the year of observation of that land use type.
non_forest_years = ee.Image()
for yr in range(1985, last_year):
    year = f"{yr}"
    lu_yr = lulc.select(year).gt(3).multiply(yr)
    non_forest_years = non_forest_years.addBands(lu_yr)

# find the minimum and maximum non-zero value per pixel across all bands
# (first and last year of anthropogenic land use)
first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())
last_anthro = non_forest_years.reduce(ee.Reducer.max())
# get total of continuous years from the first and last observation of anthropogenic land use
anthro_gaps = last_anthro.subtract(first_anthro).add(1)


In [ ]:
# Establishing a specific range for land use history for analysis
if history_interval > 0:
    lulc = lulc.select("1985").eq(3).selfMask()
    first_year_considered = 1984 + history_interval
    age = age.updateMask(age.lte(last_year - first_year_considered))
    years_regrowth_considered = range(first_year_considered, last_year)

    # Calculate the cutoff year for each pixel
    cutoff_year = ee.Image.constant(last_year + 1).subtract(age).subtract(ee.Number(history_interval))
    years = ee.Image.constant(list(range(1985, last_year + 1))).rename(lulc.bandNames())
    mask = years.gte(cutoff_year)
    lulc = lulc.updateMask(mask)

else:
    years_regrowth_considered = config.range_1985_2019
    first_year_considered = 1985


### Fallow period length

Finds the total number of years that land was left fallow (classified as forest before regrowth)

In [45]:
nat_cover = lulc.eq(3)
nat_cover = nat_cover.reduce(ee.Reducer.sum())

if history_interval > 0:

    non_forest_years = ee.Image()
    for yr in range(1985, last_year):
        year = f"{yr}"
        lu_yr = lulc.select(year).gt(3).multiply(yr)
        non_forest_years = non_forest_years.addBands(lu_yr)

    # find the minimum and maximum non-zero value per pixel across all bands
    # (first and last year of anthropogenic land use)
    first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())
    forest_before_first_anthro = first_anthro.subtract(cutoff_year)
    fallow = nat_cover.subtract(forest_before_first_anthro).subtract(age).rename("fallow").byte()
else:
    forest_before_first_anthro = first_anthro.subtract(1985)
    fallow = nat_cover.subtract(forest_before_first_anthro).subtract(age).rename("fallow").byte()

### Total sum of years under each desired land use type

For each desired land use type, the total occurrences per pixel are summed

In [46]:
lu_sum = ee.Image()
for val in lu_index:
    lulc_val = lulc.eq(val)
    num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f"lu_sum_{val}")
    lu_sum = lu_sum.addBands(num_cells)

# removes the first (empty) band
lu_sum = lu_sum.slice(1).byte()

### Last Land Use Type

Finds what was the last land use type observed before abandonment.

In [47]:
# for each forest age, mask the lulc of the year immediately preceding abandonment
last_lu = ee.Image()
for yr in config.range_1986_2019:
    year = f"{yr}"
    lu_yr = lulc.select(year)
    age_mask = age.eq(config.last_year - yr)
    last_lu = last_lu.addBands(lu_yr.updateMask(age_mask))

# remove first empty band
# merge all images by adding them up
last_lu = last_lu.slice(1).reduce(ee.Reducer.sum()).rename("last_lu").byte()

In [48]:
land_use = last_lu.addBands([fallow, lu_sum])
export_image(land_use, f"land_use{name_lu}", region = roi, scale = 30)